# Supervised Learning
## k-Nearest Neighbour Demo
In the unit material we saw a demo of how we could classify dogs and cats based on weight and tail length data. Here it is again:

| class | weight | tail length |
|:-----:|:------:|:-----------:|
|  dog  |   25   |      20     |
|  dog  |   65   |      45     |
|  dog  |    3   |      5      |
|  dog  |   35   |      30     |
|  cat  |    5   |      25     |
|  cat  |    3   |      20     |
|  cat  |    7   |      46     |

When we actually implement supervised learning algorithms we need this data in a usable form. Words like 'dog' and 'cat' are not very useful or relevant to the actual algorithm, so we assign each class a *label*, normally an integer. Let's say 'dog' is represented by `1` and cat is represented as `0`.

Now our entire table is made of integers. Since it is a table, we store it in the corresponding data structure. A programmer might call it a 2D array. A mathematician or theoretical computer scientist might call it a matrix. numpy calls everything an array!

In [1]:
import numpy as np

training_data = np.array([[1, 25, 20], 
                          [1, 65, 45],
                          [1, 3, 5],
                          [1, 35, 30],
                          [0, 5, 25],
                          [0, 3, 20],
                          [0, 7, 46]])

print(f"The shape of the training data is {training_data.shape}")
print(f"Row 3 is {training_data[3]}")
print(f"Row 3 has weight {training_data[3][1]}")

The shape of the training data is (7, 3)
Row 3 is [ 1 35 30]
Row 3 has weight 35


Our training data is a $7 \times 3$ matrix. It contains the *response variables* (the class labels in this case) in the first column. We might also decide to separate these, giving one array of size $7 \times 1$ of labels and one of size $7 \times 2$ of data.

In [2]:
# the : in an expression means means "all"
# so m[:, 0] means all rows, column zero 
# i.e. this gets the first column of matrix m
# remember: always "row by column"!

training_labels = training_data[:, 0]
training_inputs = training_data[:, 1:]

Given a new query point, the k-Nearest Neighbour algorithm finds the distance from the query point to every point in the training set. It then selects the k closest points, the class of each point can be considered a single vote, and whatever class gets the most votes is used to classify the point. Normally odd values for k are used to avoid ties. In more advanced versions the votes could be weighted by their distance from the query.

There is a very simple implementation of this below. It's been written mostly with for loops and could be made much more efficient and elegant with clever use of numpy inbuilt functions. See if you can follow it, and think about ways you could make it more efficient.

In [3]:
def knn_predict(query, training_data, k = 1):
    training_labels = training_data[:, 0]
    training_inputs = training_data[:, 1:]
    
    if query.shape[0] != training_inputs.shape[1]:
        raise KeyValueError("query point does not have correct shape")
    
    # find the distance from query to every row in training_inputs
    distances = np.zeros(training_inputs.shape[0])
    for i, training_point in enumerate(training_inputs):
        # find the Euclidean distance
        total_sum = 0
        for dim in range(0, training_point.shape[0]):
            total_sum += (training_point[dim] - query[dim]) ** 2
        distances[i] = np.sqrt(total_sum)
    
    # take the points with the k lowest distances
    # np.argsort returns the indices of each element in sorted order
    # but we could have used more for loops instead!
    sorted_indices = np.argsort(distances)
    k_closest_indices = sorted_indices[:k]
    k_closest_classes = training_labels[k_closest_indices]
    
    # now find the most common element
    votes_for_1 = np.count_nonzero(k_closest_classes == 1)
    
    if votes_for_1 > k/2:
        return 1
    else:
        return 0
    
query = np.array([40, 30])
prediction = knn_predict(query, training_data)
print(f"Predict that {query} is class {prediction} (k=1)")

query = np.array([20, 20])
prediction = knn_predict(query, training_data)
print(f"Predict that {query} is class {prediction} (k=1)")

query = np.array([20, 20])
prediction = knn_predict(query, training_data, k = 3)
print(f"Predict that {query} is class {prediction} (k=3)")

Predict that [40 30] is class 1 (k=1)
Predict that [20 20] is class 1 (k=1)
Predict that [20 20] is class 0 (k=3)


Remember that kNN extends nicely into higher dimensions, it is just harder to visualise what it means to take the distance between two points in 5 or 20 dimensions.

The next cell loads some data about animals, based on some data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Zoo). The idea is to try to classify whether the animal is a mammal or not based on various attributes, all Boolean: whether it has feathers, whether it is toothed, whether it has a tail, whether it the same size as a cat, and so on. I've already stripped out the attributes that would completely give the game away, like whether it produces milk!

This time we will treat the data properly by separating it into two groups, one to use as the training data for kNN, and the rest to use as test data to see how well it performs. See if you can follow the code below.

In [4]:
np.random.seed(4)

zoo_all_data = np.loadtxt('data/zoo.csv', delimiter=',', dtype=np.object)
print(f"The attributes are: \n{zoo_all_data[0, :]}")
zoo_all_data = zoo_all_data[1:, :]
np.random.shuffle(zoo_all_data)
print("Preview of the data:")
print(zoo_all_data)
print(f"zoo_all_data has shape {zoo_all_data.shape}")

The attributes are: 
['animal_name' 'mammal' 'feathers' 'airborne' 'aquatic' 'predator'
 'toothed' 'backbone' 'breathes' 'venomous' 'fins' 'tail' 'domestic'
 'catsize']
Preview of the data:
[['dove' '0' '1' ... '1' '1' '0']
 ['cheetah' '1' '0' ... '1' '0' '1']
 ['wasp' '0' '0' ... '0' '0' '0']
 ...
 ['raccoon' '1' '0' ... '1' '0' '1']
 ['oryx' '1' '0' ... '1' '0' '1']
 ['lobster' '0' '0' ... '0' '0' '0']]
zoo_all_data has shape (101, 14)


C:\Users\ALEXDR~1\AppData\Local\Temp/ipykernel_22476/3272119399.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  zoo_all_data = np.loadtxt('data/zoo.csv', delimiter=',', dtype=np.object)


In [38]:
# split data into training and testing, 70 training and 31 testing
zoo_training_data = zoo_all_data[:70, :]
zoo_testing_data = zoo_all_data[70:, :]

# split the names (not used in the actual classification), labels, and other variables
zoo_testing_names = zoo_testing_data[:, 0]
zoo_testing_labels = zoo_testing_data[:, 1].astype(np.int)
zoo_testing_inputs = zoo_testing_data[:, 2:].astype(np.int)

# split the names off the training data too (but leave labels)
zoo_training = zoo_training_data[:, 1:].astype(np.int)

# now go through each of the testing set and see how many kNN predicts correctly
score = 0
for i in range(0, zoo_testing_inputs.shape[0]):
    test_input = zoo_testing_inputs[i, :]
    test_correct_label = zoo_testing_labels[i]
    test_name = zoo_testing_names[i]
    
    prediction = knn_predict(test_input, zoo_training, k = 3)
    
    if prediction == test_correct_label:
        print(f"+ {test_name} correctly classified as {prediction}")
        score += 1
    else:
        print(f"- {test_name} incorrectly classified as {prediction}")
        
accuracy = score/zoo_testing_data.shape[0]
print(f"\nTotal accuracy is {accuracy}")

+ sealion correctly classified as 1
+ mole correctly classified as 1
+ catfish correctly classified as 0
+ gnat correctly classified as 0
+ wren correctly classified as 0
+ porpoise correctly classified as 1
+ ostrich correctly classified as 0
+ starfish correctly classified as 0
+ seahorse correctly classified as 0
- slowworm incorrectly classified as 1
+ duck correctly classified as 0
- tortoise incorrectly classified as 1
+ aardvark correctly classified as 1
+ bear correctly classified as 1
- newt incorrectly classified as 1
+ herring correctly classified as 0
+ leopard correctly classified as 1
+ vole correctly classified as 1
+ hare correctly classified as 1
+ parakeet correctly classified as 0
- tuatara incorrectly classified as 1
+ worm correctly classified as 0
- penguin incorrectly classified as 1
+ cavy correctly classified as 1
+ mongoose correctly classified as 1
+ scorpion correctly classified as 0
+ swan correctly classified as 0
+ antelope correctly classified as 1
+ rac

The total accuracy looks to be about 84%, which doesn't seem too bad for such a simple algorithm. On its own that's useful to know, but the real value of testing your algorithm is so that you can compare different approaches, or to help you pick the parameters. Here we used `k=3`, but maybe we would get better results using 1-nearest neighbour or 5-nearest neighbour?

As it happens, this dataset gets the same accuracy for all reasonable values of k, but for a larger problem, this is where you could try the various settings before picking which one is best. Then, once you are done, you can use all of the data to form the final model – in this case that would simply mean using all of the data during the kNN algorithm.

You might also want to make better use of all of the data, perhaps the 30% we picked here just happen to be particularly difficult examples? We can get a better estimate of how the entire dataset will perform if we perform multiple rounds of *k-fold cross validation*. Refer to the unit material to learn more.